<a href="https://colab.research.google.com/github/ApplesDaddy/CMPT353Project/blob/main/regression_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!conda --version
!which python
!python --version
!echo $PYTHONPATH

'conda' is not recognized as an internal or external command,
operable program or batch file.
'which' is not recognized as an internal or external command,
operable program or batch file.


Python 3.11.4
$PYTHONPATH


In [2]:
%env PYTHONPATH=
!echo $PYTHONPATH

env: PYTHONPATH=
$PYTHONPATH


In [3]:
# most recent install link from https://docs.anaconda.com/free/miniconda/miniconda-other-installer-links/
!wget https://repo.anaconda.com/miniconda/Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!chmod +x Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!bash ./Miniconda3-py310_24.1.2-0-Linux-x86_64.sh -b -f -p /usr/local/

'wget' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.
/bin/bash: ./Miniconda3-py310_24.1.2-0-Linux-x86_64.sh: No such file or directory


In [4]:
!which conda
!conda --version
!conda install --channel defaults conda python=3.10 --yes
!conda update --channel defaults --all --yes

'which' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import sys
sys.path

['C:\\Users\\17788\\OneDrive - Simon Fraser University (1sfu)\\Desktop\\School\\CMPT353\\Project\\CMPT353Project',
 'C:\\Users\\17788\\AppData\\Local\\Programs\\Python\\Python311',
 'C:\\Users\\17788\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip',
 'C:\\Users\\17788\\AppData\\Local\\Programs\\Python\\Python311\\DLLs',
 'C:\\Users\\17788\\AppData\\Local\\Programs\\Python\\Python311\\Lib',
 '',
 'C:\\Users\\17788\\AppData\\Roaming\\Python\\Python311\\site-packages',
 'C:\\Users\\17788\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages',
 'C:\\Users\\17788\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\win32',
 'C:\\Users\\17788\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\17788\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\Pythonwin']

In [6]:
!ls /usr/local/lib/python3.10/dist-packages

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
sys.path.append("/usr/local/lib/python3.10/site-packages")
!conda config --add channels conda-forge

'conda' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
!conda create --name musemotion_meta -c conda-forge numpy pandas matplotlib scikit-learn wandb pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

'conda' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
# !conda init
# !conda activate musemotion_meta
# credit https://stackoverflow.com/a/62668276
%%shell
eval "$(conda shell.bash hook)"
conda activate musemotion_meta

SyntaxError: invalid syntax (1993114121.py, line 5)

# Load data
_Note_: tempo = bpm

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://github.com/zhandrew4/music-mood-classification/raw/main/muse_v3.csv", "datasets/muse_v3.csv")
urllib.request.urlretrieve("https://github.com/zhandrew4/music-mood-classification/raw/main/full_data.csv", "datasets/full_data.csv")

In [ ]:
import pandas as pd
sample = pd.read_csv("datasets/full_data.csv", nrows=3)
sample.columns

In [ ]:
KEEP_COLS_MASTER = ["track", "album", "artist", "release_date", "length",
             "tempo", "key", "time_signature",
             "mood"]

KEEP_COLS_MODEL = ["track", "artist", "tempo", "key", "loudness", "time_signature", "duration_ms"]

# songs_df = pd.read_csv("data_moods.csv", usecols=KEEP_COLS_MASTER)
# make names the indices
songs_df = pd.read_csv("datasets/full_data.csv", usecols=KEEP_COLS_MODEL, parse_dates=True, infer_datetime_format=True)
songs_df.set_index("track", inplace=True)
songs_df

Convert dates to numbers, specfically seconds.

_Note_: This goddamn dataset only gives the year for some songs.

In [ ]:
# print(songs_df[songs_df["release_date"]=="1993"])
# # credit https://stackoverflow.com/a/54312941
# songs_df["release_date"] = pd.to_datetime(songs_df["release_date"], format="mixed").astype(int)/ 10**9
# songs_df["release_date"]

Load the V, A, D dimension values from the original MuSe dataset and add them to the rest of the data.
^ Join V, A, D columns by key columns title and author.

In [ ]:
import pandas as pd

orig_dataset = pd.read_csv("datasets/muse_v3.csv")
orig_dataset.columns
orig_dataset.set_index("track", inplace=True)

In [ ]:
VAL_COLS = ["valence_tags","arousal_tags","dominance_tags"]
songs_df = songs_df.join(orig_dataset[VAL_COLS], how="inner")

Model will take data as (`X_train`,`y_train`)
- `X_train` is a 2D NumPy array of shape `n_samples` x `n_features`, i.e. one song per row, one metadata feature per column
- `y_train` is just convert the "mood" column into a 1D NumPy array

In [ ]:
# How much should we take for training, testing?
songs_df.index.size

0.7 * songs_df.index.size

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

DROP_COLS = VAL_COLS+["artist"]
# use a constant random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(songs_df.drop(columns=DROP_COLS), songs_df[VAL_COLS],
                                                    test_size=0.2, random_state=42)

# Model
## Multi-Layer Perceptron
[PyTorch tutorial](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

[Building a Regression Model in PyTorch](https://machinelearningmastery.com/building-a-regression-model-in-pytorch/)

[ChatGPT Explaining The PyTorch Tutorial...](https://chat.openai.com/share/424f5717-901e-4dac-9f71-f30b1e4ab6de)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(solver='lbfgs', hidden_layer_sizes=())


mlp_regression = mlp.fit(X_train, y_train)
print(mlp_regression.score(X_test, y_test))

In [ ]:
# import os
# import torch
# from torch import nn
# from torch.utils.data import DataLoader
# from torchvision import datasets, transforms

In [ ]:
# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )
# print(f"Using {device} device")

In [ ]:
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(3, 1),
#             nn.ReLU(),
#             nn.Linear(3, 1),
#             nn.ReLU(),
#             nn.Linear(3, 1),
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits

In [ ]:
# model = NeuralNetwork().to(device)
# print(model)

## Histogram-Based Gradient Boosted Tree
- [scikit-learn tutorial](https://scikit-learn.org/stable/modules/ensemble.html#histogram-based-gradient-boosting)
- [regressor API reference](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#sklearn.ensemble.HistGradientBoostingRegressor)

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_validate

hgbt = MultiOutputRegressor(HistGradientBoostingRegressor(random_state=0))
regression = hgbt.fit(X_train, y_train)

In [ ]:
regression.score(X_test, y_test)